In [1]:
import time
import os

# import scanpy
import numpy as np
import scipy.sparse as sp

import torch
from torch import optim
from torch.utils.data import DataLoader

import models.modelsCNN as modelsCNN
import models.optimizer as optimizer

import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import gc
from skimage import io
import scipy.stats
import utils.plot
import models.train_metaClf

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

resolution='5'
savedir_baseline='/data/xzhang/neuro/results/jointClustering/baseline/res'+resolution
with open(os.path.join(savedir_baseline,'pca_dapi_600'), 'rb') as output:
    pca_dapi=pickle.load(output)
with open(os.path.join(savedir_baseline,'pca_gfap_600'), 'rb') as output:
    pca_gfap=pickle.load(output)
with open(os.path.join(savedir_baseline,'pca_map2_600'), 'rb') as output:
    pca_map2=pickle.load(output)
with open(os.path.join(savedir_baseline,'pca_lmnb_600'), 'rb') as output:
    pca_lmnb=pickle.load(output)
with open(os.path.join(savedir_baseline,'modAll'), 'rb') as output:
    modAll=pickle.load(output)
with open(os.path.join(savedir_baseline,'silAll'), 'rb') as output:
    silAll=pickle.load(output)
with open(os.path.join(savedir_baseline,'moddapi'), 'rb') as output:
    moddapi=pickle.load(output)
with open(os.path.join(savedir_baseline,'sildapi'), 'rb') as output:
    sildapi=pickle.load(output)
with open(os.path.join(savedir_baseline,'modgfap'), 'rb') as output:
    modgfap=pickle.load(output)
with open(os.path.join(savedir_baseline,'silgfap'), 'rb') as output:
    silgfap=pickle.load(output)
with open(os.path.join(savedir_baseline,'modmap2'), 'rb') as output:
    modmap2=pickle.load(output)
with open(os.path.join(savedir_baseline,'silmap2'), 'rb') as output:
    silmap2=pickle.load(output)
with open(os.path.join(savedir_baseline,'modlmnb'), 'rb') as output:
    modlmnb=pickle.load(output)
with open(os.path.join(savedir_baseline,'sillmnb'), 'rb') as output:
    sillmnb=pickle.load(output)
with open(os.path.join(savedir_baseline,'leidenRes3_10samples'), 'rb') as output:
    leidenResAll=pickle.load(output)
    
savedir_processed='/data/xzhang/neuro/processed'
resDir='/data/xzhang/neuro/results/plots/cnnvaeexp0_segNucall'
saveDir=os.path.join(savedir_baseline,'cnnvaeexp0_segNucall')
if not os.path.exists(saveDir):
    os.mkdir(saveDir)


with open(os.path.join(resDir,'leiden_res'+resolution), 'rb') as output:
    finalPartition=pickle.load(output)
    
jointPartition=np.copy(finalPartition)
modFrac=0.3
modFrac_dapi=(moddapi/modAll>modFrac).astype(int)
modFrac_map2=(modmap2/modAll>modFrac).astype(int)
modFrac_gfap=(modgfap/modAll>modFrac).astype(int)
modFrac_lmnb=(modlmnb/modAll>modFrac).astype(int)
modFrac_sum=modFrac_dapi+modFrac_gfap+modFrac_lmnb+modFrac_map2

silAll_mean=np.zeros(np.unique(jointPartition).size)
for c in np.unique(jointPartition):
    silAll_mean[c]=np.mean(silAll[jointPartition==c])
    
sildapi_mean=np.zeros(np.unique(jointPartition).size)
for c in np.unique(jointPartition):
    sildapi_mean[c]=np.mean(sildapi[jointPartition==c])
    
silgfap_mean=np.zeros(np.unique(jointPartition).size)
for c in np.unique(jointPartition):
    silgfap_mean[c]=np.mean(silgfap[jointPartition==c])

silmap2_mean=np.zeros(np.unique(jointPartition).size)
for c in np.unique(jointPartition):
    silmap2_mean[c]=np.mean(silmap2[jointPartition==c])
sillmnb_mean=np.zeros(np.unique(jointPartition).size)
for c in np.unique(jointPartition):
    sillmnb_mean[c]=np.mean(sillmnb[jointPartition==c])
silAll_pos=np.zeros(np.unique(jointPartition).size)
for c in np.unique(jointPartition):
    silAll_pos[c]=np.sum(silAll[jointPartition==c]>0)/np.sum(jointPartition==c)
sildapi_pos=np.zeros(np.unique(jointPartition).size)
for c in np.unique(jointPartition):
    sildapi_pos[c]=np.sum(sildapi[jointPartition==c]>0)/np.sum(jointPartition==c)
silmap2_pos=np.zeros(np.unique(jointPartition).size)
for c in np.unique(jointPartition):
    silmap2_pos[c]=np.sum(silmap2[jointPartition==c]>0)/np.sum(jointPartition==c)
silgfap_pos=np.zeros(np.unique(jointPartition).size)
for c in np.unique(jointPartition):
    silgfap_pos[c]=np.sum(silgfap[jointPartition==c]>0)/np.sum(jointPartition==c)
sillmnb_pos=np.zeros(np.unique(jointPartition).size)
for c in np.unique(jointPartition):
    sillmnb_pos[c]=np.sum(sillmnb[jointPartition==c]>0)/np.sum(jointPartition==c)

silmean_frac=0.3
silmean_frac_neg=2
def comparesilmean(sildapi_mean,silAll_mean,silmean_frac,silmean_frac_neg):
    silmean_frac_dapi=np.zeros(silAll_mean.size)
    sildapi_mean_posIdx=np.arange(silAll_mean.size)[sildapi_mean>0]
    sildapi_mean_negIdx=np.arange(silAll_mean.size)[sildapi_mean<0]
#     print(sildapi_mean_negIdx[sildapi_mean[sildapi_mean<0]>(silAll_mean[sildapi_mean<0]*silmean_frac_neg)])
    silmean_frac_dapi[sildapi_mean_posIdx[sildapi_mean[sildapi_mean>0]>(silAll_mean[sildapi_mean>0]*silmean_frac)]]=1
    silmean_frac_dapi[sildapi_mean_negIdx[sildapi_mean[sildapi_mean<0]>(silAll_mean[sildapi_mean<0]*silmean_frac_neg)]]=1

    return silmean_frac_dapi

silmean_frac_dapi=comparesilmean(sildapi_mean,silAll_mean,silmean_frac,silmean_frac_neg)
silmean_frac_gfap=comparesilmean(silgfap_mean,silAll_mean,silmean_frac,silmean_frac_neg)
silmean_frac_map2=comparesilmean(silmap2_mean,silAll_mean,silmean_frac,silmean_frac_neg)
silmean_frac_lmnb=comparesilmean(sillmnb_mean,silAll_mean,silmean_frac,silmean_frac_neg)

silmean_frac_sum=silmean_frac_dapi+silmean_frac_gfap+silmean_frac_map2+silmean_frac_lmnb

silpos_frac=0.3
silpos_dapi=(sildapi_pos/silAll_pos>silpos_frac).astype(int)
silpos_gfap=(silgfap_pos/silAll_pos>silpos_frac).astype(int)
silpos_map2=(silmap2_pos/silAll_pos>silpos_frac).astype(int)
silpos_lmnb=(sillmnb_pos/silAll_pos>silpos_frac).astype(int)

silPos_sum=silpos_dapi+silpos_gfap+silpos_map2+silpos_lmnb


clusterIdx_pass=np.arange(np.unique(finalPartition).size)[(modFrac_sum>1)+(silmean_frac_sum>1)+(silPos_sum>1)]

nsamples=10
leidenJoint_overlap=np.zeros((np.unique(jointPartition).size,nsamples))
for cj in np.unique(jointPartition):
    for s in range(nsamples):
        _,counts=np.unique(leidenResAll[jointPartition==cj,s],return_counts=True)
        leidenJoint_overlap[cj,s]=np.max(counts)/np.sum(jointPartition==cj)

overlap_mean=np.mean(leidenJoint_overlap,axis=1)
overlap_std=np.std(leidenJoint_overlap,axis=1)

overlap_mean=np.mean(leidenJoint_overlap,axis=1)
overlap_std=np.std(leidenJoint_overlap,axis=1)

randomAll=np.zeros((leidenResAll.shape[0],nsamples)).astype(int)

for s in range(nsamples):
    randomS=np.copy(jointPartition)
    np.random.seed(s)
    np.random.shuffle(randomS)

    randomAll[:,s]=randomS
leidenJoint_overlap_random=np.zeros((np.unique(jointPartition).size,nsamples))
for cj in np.unique(jointPartition):
    for s in range(nsamples):
        _,counts=np.unique(randomAll[jointPartition==cj,s],return_counts=True)
        leidenJoint_overlap_random[cj,s]=np.max(counts)/np.sum(jointPartition==cj)

overlap_mean_random=np.mean(leidenJoint_overlap_random,axis=1)
overlap_std_random=np.std(leidenJoint_overlap_random,axis=1)

clusterIdx_robust=np.arange(np.unique(finalPartition).size)[(overlap_mean-overlap_std)>(overlap_mean_random+overlap_std_random)]
finalPartition_keep=np.intersect1d(clusterIdx_robust,clusterIdx_pass)


    
with open(os.path.join(savedir_processed,'meta_processed'), 'rb') as output:
    meta=pickle.load(output)

# annoUsed=np.array(['AAO', 'AAD', 'Duration', 'Gender','Clinical Diag','Brain Weight', 'Mutations',
#                    'APOE', 'Braak Tau', 'Thal Phase', 'CERAD', 'CAA', 'SVD','alpha-syn', 'TDP43'])
annoUsed=np.array(['Path Diag','Clinical Diag','Brain Weight', 'Mutations',
                   'APOE', 'Braak Tau', 'Thal Phase', 'CERAD', 'CAA', 'SVD','alpha-syn', 'TDP43',
                   'AAO', 'AAD', 'Duration', 'Gender'])
annoUsed_numeric=np.array(['AAO', 'AAD', 'Duration','Brain Weight','Braak Tau', 'Thal Phase', 'CERAD', 'CAA', 'SVD'])

meta=meta[annoUsed]
stains=np.array(['DAPI','GFAP','MAP2','LMNB'])

    
finalPartition_keepIdx=np.repeat(False, finalPartition.size)
for c in finalPartition_keep:
    finalPartition_keepIdx[finalPartition==c]=True
finalPartition_keep_input=np.copy(finalPartition)
finalPartition_keep_input[np.logical_not(finalPartition_keepIdx)]=-1



In [3]:
with open(os.path.join(savedir_processed,'allPatientID'), 'rb') as output:
    allPatientIDs=pickle.load(output)
meta=meta.loc[np.unique(allPatientIDs)]

In [4]:
np.unique(finalPartition_keep).size

68

In [5]:
with open(os.path.join(savedir_processed,'allImgNames'), 'rb') as output:
    allImgNames=pickle.load(output)
print(np.unique(allImgNames).size)
print(np.unique(allPatientIDs).size)

64
50


In [6]:
spatialRegions=[300,650,1000]

In [7]:
#get cluster composition
sUnique_all=np.unique(allImgNames)
inputDict={}
clusterRes_i=finalPartition_keep_input[finalPartition_keepIdx]
clusterUnique=np.unique(clusterRes_i)
for sr in spatialRegions:
    with open(os.path.join(savedir_baseline,'clusterPlots','neighborhood_allCells_counts_'+str(sr)), 'rb') as output:
        neigh=pickle.load(output)
    inputCluster=np.zeros((sUnique_all.size,clusterUnique.size))
    for i in range(sUnique_all.size):
        imgIdx_i=allImgNames[finalPartition_keepIdx]==sUnique_all[i]
        for j in range(clusterUnique.size):
            cellIdx=np.logical_and(clusterRes_i==clusterUnique[j],imgIdx_i)
            if np.sum(cellIdx)>0:
                inputCluster[i,j]=np.mean(neigh[cellIdx])
    inputDict[sr]=inputCluster


In [8]:
inputDict[sr].shape

(64, 68)

In [ ]:
seeds=[4,8,9,10,11,12]
for anno in ['APOE']:
    print(anno)
    regrs=False
    if anno in annoUsed_numeric:
        regrs=True
    if anno in annoUsed_numeric:
        continue
        
    
    #construct labels
    
    meta['keep']=True
    if anno not in annoUsed_numeric:
        pathDiag_unique_keep,pathDiag_inverse_keep,pathDiag_counts_keep=np.unique(meta[anno].astype(str),return_counts=True,return_inverse=True)
        pathDiag_unique_keep[pathDiag_counts_keep<=1]='remove'
        pathDiag_unique_keep=pathDiag_unique_keep[pathDiag_inverse_keep]

        meta['keep']=pathDiag_unique_keep!='remove'
    
    meta_allcells=meta.loc[allPatientIDs][anno].to_numpy().astype(str)
    cellIdx=np.logical_and(meta_allcells!='na',meta_allcells!='nan')
    cellIdx=np.logical_and(cellIdx,meta.loc[allPatientIDs]['keep'].to_numpy())
    allCat=meta_allcells[cellIdx]
    
    allImgNames_a=allImgNames[cellIdx]
    sUnique,sidx_start=np.unique(allImgNames_a,return_index=True)
    

    if anno not in annoUsed_numeric:
        pathDiag_unique,labels,pathDiag_counts=np.unique(allCat[sidx_start],return_counts=True,return_inverse=True)
        weights=np.sum(pathDiag_counts)/pathDiag_counts
    else:
        pathDiag_unique=None
        weights=None
        labels=allCat[sidx_start].astype(float)
    if labels.size<2:
        continue
    annoName=anno
    if ' ' in anno:
        annoName=anno.split(' ')
        annoName=annoName[0]+'_'+annoName[1]
    for seed in seeds:
        print('seed: ',seed)
        for sr in spatialRegions:
            print('spatial ',sr)
            _,idx,_=np.intersect1d(sUnique_all,sUnique,return_indices=True)
            inputAll=inputDict[sr][idx]
            name='exp0_allMetaClf_nucSeg_modSilFilter_2_res5_seed'+str(seed)+'_'+annoName+'_'+str(sr)
            savepath_curr=os.path.join(saveDir,'metadataClf_counts',name)
            if not os.path.exists(os.path.join(saveDir,'metadataClf_counts')):
                os.mkdir(os.path.join(saveDir,'metadataClf_counts'))
            if not os.path.exists(savepath_curr):
                os.mkdir(savepath_curr)
            logsavepath=os.path.join(savepath_curr,'log')
            modelsavepath=os.path.join(savepath_curr,'model')
            plotsavepath=os.path.join(savepath_curr,'plots')

    #         if os.path.exists(os.path.join(plotsavepath,'predictions'+str(5900)+'.csv')):
    #             continue

            if not os.path.exists(logsavepath):
                os.mkdir(logsavepath)
            if not os.path.exists(modelsavepath):
                os.mkdir(modelsavepath)
            if not os.path.exists(plotsavepath):
                os.mkdir(plotsavepath)

            pIDList=allPatientIDs[sidx_start]
            models.train_metaClf.train_metaClf(inputAll, pathDiag_unique, labels, logsavepath, modelsavepath, plotsavepath, pIDList, allImgNames, sidx_start,weights=weights, use_cuda=True, seed=seed, testepoch=3500, epochs=4000, saveFreq=500, lr=0.001, weight_decay=0, batchsize=32, model_str='fc3', fc_dim=256,regrs=regrs,update=True)

In [52]:
#plot density
plotsavepath='/data/xzhang/neuro/results/jointClustering/baseline/res5/cnnvaeexp0_segNucall/metadataClf_counts/plots'
if not os.path.exists(plotsavepath):
    os.mkdir(plotsavepath)

In [54]:
clusterRes_i=finalPartition_keep_input[finalPartition_keepIdx]
clusterUnique=np.unique(clusterRes_i)
anno='APOE'
meta['keep']=True
if anno not in annoUsed_numeric:
    pathDiag_unique_keep,pathDiag_inverse_keep,pathDiag_counts_keep=np.unique(meta[anno].astype(str),return_counts=True,return_inverse=True)
    pathDiag_unique_keep[pathDiag_counts_keep<=1]='remove'
    pathDiag_unique_keep=pathDiag_unique_keep[pathDiag_inverse_keep]

    meta['keep']=pathDiag_unique_keep!='remove'
for sr in spatialRegions:
    
    meta_allcells=meta.loc[allPatientIDs][anno].to_numpy().astype(str)
    cellIdx=np.logical_and(meta_allcells!='na',meta_allcells!='nan')
    cellIdx=np.logical_and(cellIdx,meta.loc[allPatientIDs]['keep'].to_numpy())
    allCat=meta_allcells[cellIdx]

    allImgNames_a=allImgNames[cellIdx]
    sUnique,sidx_start=np.unique(allImgNames_a,return_index=True)

    pathDiag_unique,labels,pathDiag_counts=np.unique(allCat[sidx_start],return_counts=True,return_inverse=True)
    _,idx,_=np.intersect1d(sUnique_all,sUnique,return_indices=True)
    inputAll=inputDict[sr][idx]
    input_pos=inputAll[labels==1]
    input_neg=inputAll[labels==0]
    input_plot=np.zeros((inputAll.shape[1],2))
    input_plot[:,0]=np.mean(input_neg,axis=0)
    input_plot[:,1]=np.mean(input_pos,axis=0)

    vmax=np.max(input_plot)
    vmin=np.median(input_plot)-(np.max(input_plot)-np.median(input_plot))
    figsize=(3,15)
    fig, ax = plt.subplots(figsize=figsize)
    im = ax.imshow(input_plot,cmap='bwr',vmin=vmin,vmax=vmax)
    ax.set_yticks(np.arange(input_plot.shape[0]))
    ax.set_yticklabels(clusterUnique)
    ax.set_xticks(np.arange(input_plot.shape[1]))
    ax.set_xticklabels([0,1])
    plt.setp(ax.get_xticklabels(), rotation=90, ha="right",rotation_mode="anchor")
    fig.colorbar(im)
    fig.tight_layout()
    plt.savefig(os.path.join(plotsavepath,anno+'_'+str(sr)+'.pdf'))
    plt.close()

In [36]:
input_plot.shape

(30, 68)

In [45]:
clusterRes_i=finalPartition_keep_input[finalPartition_keepIdx]
clusterUnique=np.unique(clusterRes_i)
anno='APOE'
meta['keep']=True
if anno not in annoUsed_numeric:
    pathDiag_unique_keep,pathDiag_inverse_keep,pathDiag_counts_keep=np.unique(meta[anno].astype(str),return_counts=True,return_inverse=True)
    pathDiag_unique_keep[pathDiag_counts_keep<=1]='remove'
    pathDiag_unique_keep=pathDiag_unique_keep[pathDiag_inverse_keep]

    meta['keep']=pathDiag_unique_keep!='remove'
for sr in spatialRegions:
    
    meta_allcells=meta.loc[allPatientIDs][anno].to_numpy().astype(str)
    cellIdx=np.logical_and(meta_allcells!='na',meta_allcells!='nan')
    cellIdx=np.logical_and(cellIdx,meta.loc[allPatientIDs]['keep'].to_numpy())
    allCat=meta_allcells[cellIdx]

    allImgNames_a=allImgNames[cellIdx]
    sUnique,sidx_start=np.unique(allImgNames_a,return_index=True)

    pathDiag_unique,labels,pathDiag_counts=np.unique(allCat[sidx_start],return_counts=True,return_inverse=True)
    _,idx,_=np.intersect1d(sUnique_all,sUnique,return_indices=True)
    inputAll=inputDict[sr][idx]
    input_pos=inputAll[labels==1]
    input_neg=inputAll[labels==0]
    input_plot=np.concatenate((input_pos,input_neg),axis=0)

    vmax=np.max(input_plot)
    vmin=np.median(input_plot)-(np.max(input_plot)-np.median(input_plot))
    figsize=(15,6)
    fig, ax = plt.subplots(figsize=figsize)
    im = ax.imshow(input_plot,cmap='bwr',vmin=vmin,vmax=vmax)
    ax.set_xticks(np.arange(input_plot.shape[1]))
    ax.set_xticklabels(clusterUnique)
    ax.set_yticks([0,np.sum(labels==1)])
    ax.set_yticklabels([pathDiag_unique[1],pathDiag_unique[0]])
    plt.setp(ax.get_xticklabels(), rotation=90, ha="right",rotation_mode="anchor")
    fig.colorbar(im)
    fig.tight_layout()
    plt.savefig(os.path.join(plotsavepath,anno+'_'+str(sr)+'_all.pdf'))
    plt.close()

In [58]:
with open('/data/xzhang/neuro/results/jointClustering/baseline/res5/clusterPlots/ls_cc_level1Val', 'rb') as output:
    ls_cc_fin_clusters=pickle.load(output)
cnames,corder=np.unique(ls_cc_fin_clusters[1000],return_inverse=True)

clusterRes_i=finalPartition_keep_input[finalPartition_keepIdx]
clusterUnique=np.unique(clusterRes_i)
anno='APOE'
meta['keep']=True
if anno not in annoUsed_numeric:
    pathDiag_unique_keep,pathDiag_inverse_keep,pathDiag_counts_keep=np.unique(meta[anno].astype(str),return_counts=True,return_inverse=True)
    pathDiag_unique_keep[pathDiag_counts_keep<=1]='remove'
    pathDiag_unique_keep=pathDiag_unique_keep[pathDiag_inverse_keep]

    meta['keep']=pathDiag_unique_keep!='remove'
for sr in spatialRegions:
    meta_allcells=meta.loc[allPatientIDs][anno].to_numpy().astype(str)
    cellIdx=np.logical_and(meta_allcells!='na',meta_allcells!='nan')
    cellIdx=np.logical_and(cellIdx,meta.loc[allPatientIDs]['keep'].to_numpy())
    allCat=meta_allcells[cellIdx]

    allImgNames_a=allImgNames[cellIdx]
    sUnique,sidx_start=np.unique(allImgNames_a,return_index=True)

    pathDiag_unique,labels,pathDiag_counts=np.unique(allCat[sidx_start],return_counts=True,return_inverse=True)
    _,idx,_=np.intersect1d(sUnique_all,sUnique,return_indices=True)
    inputAll=inputDict[sr][idx]
    input_pos=inputAll[labels==1]
    input_neg=inputAll[labels==0]
    input_plot=np.zeros((inputAll.shape[1],2))
    input_plot[:,0]=np.mean(input_neg,axis=0)
    input_plot[:,1]=np.mean(input_pos,axis=0)

    vmax=np.max(input_plot)
    vmin=np.median(input_plot)-(np.max(input_plot)-np.median(input_plot))
    figsize=(3,15)
    fig, ax = plt.subplots(figsize=figsize)
    im = ax.imshow(input_plot[corder],cmap='bwr',vmin=vmin,vmax=vmax)
    ax.set_yticks(np.arange(input_plot.shape[0]))
    ax.set_yticklabels(clusterUnique[corder])
    ax.set_xticks(np.arange(input_plot.shape[1]))
    ax.set_xticklabels([0,1])
    plt.setp(ax.get_xticklabels(), rotation=90, ha="right",rotation_mode="anchor")
    fig.colorbar(im)
    fig.tight_layout()
    plt.savefig(os.path.join(plotsavepath,anno+'_'+str(sr)+'Ordered.pdf'))
    plt.close()

### integrated gradients

In [1]:
from scipy import stats
import sys
sys.path.append('/home/xzhang/anaconda3/envs/neuro/lib/python3.10/site-packages/')
from captum.attr import IntegratedGradients

#### igResControl
-> use controls as baselines

In [24]:
seeds=[4,8,9,10,11,12]
annoPlot_cat=np.array(['APOE','TDP43'])
annoPlot_baseline={'Path Diag':'Control',
                   'Clinical Diag':'Control',
                   'APOE':'33',
                   'TDP43':'0.0',
                   'alpha-syn':'0.0'}

for anno in annoPlot_cat:
    print(anno)
    regrs=False
        
    
    #construct labels
    
    meta['keep']=True
    if anno not in annoUsed_numeric:
        pathDiag_unique_keep,pathDiag_inverse_keep,pathDiag_counts_keep=np.unique(meta[anno].astype(str),return_counts=True,return_inverse=True)
        pathDiag_unique_keep[pathDiag_counts_keep<=1]='remove'
        pathDiag_unique_keep=pathDiag_unique_keep[pathDiag_inverse_keep]

        meta['keep']=pathDiag_unique_keep!='remove'
    
    meta_allcells=meta.loc[allPatientIDs][anno].to_numpy().astype(str)
    cellIdx=np.logical_and(meta_allcells!='na',meta_allcells!='nan')
    cellIdx=np.logical_and(cellIdx,meta.loc[allPatientIDs]['keep'].to_numpy())
    allCat=meta_allcells[cellIdx]
    
    allImgNames_a=allImgNames[cellIdx]
    sUnique,sidx_start=np.unique(allImgNames_a,return_index=True)
    
    pathDiag_unique,labels,pathDiag_counts=np.unique(allCat[sidx_start],return_counts=True,return_inverse=True)
    weights=np.sum(pathDiag_counts)/pathDiag_counts
    if labels.size<2:
        continue
    annoName=anno
    if ' ' in anno:
        annoName=anno.split(' ')
        annoName=annoName[0]+'_'+annoName[1]
        
    true_label=np.copy(pathDiag_unique[labels]).astype(str)
    for seed in seeds:
        print('seed: ',seed)
        for sr in spatialRegions:
            print('spatial ',sr)
            _,idx,_=np.intersect1d(sUnique_all,sUnique,return_indices=True)
            inputAll=inputDict[sr][idx]
            inputBaseline=inputAll[true_label==annoPlot_baseline[anno]]
            
            name='exp0_allMetaClf_nucSeg_modSilFilter_2_res5_seed'+str(seed)+'_'+annoName+'_'+str(sr)
            savepath_curr=os.path.join(saveDir,'metadataClf_counts',name)
            logsavepath=os.path.join(savepath_curr,'log')
            modelsavepath=os.path.join(savepath_curr,'model')
            plotsavepath=os.path.join(savepath_curr,'plots')

    #         if os.path.exists(os.path.join(plotsavepath,'predictions'+str(5900)+'.csv')):
    #             continue

            pIDList=allPatientIDs[sidx_start]

            use_cuda=True
            testepoch=3500
            batchsize=32
            model_str='fc3'
            fc_dim=256

            fc_dim1=fc_dim
            fc_dim2=fc_dim
            fc_dim3=fc_dim

            if regrs:
                nclasses=1
            else:
                nclasses=np.unique(labels).size

            igRes=np.zeros_like(inputAll)
            igResPred=np.zeros_like(inputAll)

            for patientIDX in range(np.unique(pIDList).size):
                patientID=np.unique(pIDList)[patientIDX]
        #         print(patientID,patientIDX)
                sampleIdx=np.arange(inputAll.shape[0])[pIDList==patientID]
                trainIdx=np.arange(inputAll.shape[0])[pIDList!=patientID]

                inputAll_t=torch.tensor(inputAll[sampleIdx],requires_grad=True).cuda().float()
                for ibIdx in range(inputBaseline.shape[0]):
                    inputBaseline_t=torch.tensor(inputBaseline[[ibIdx]],requires_grad=True).cuda().float()
        #             labels=torch.tensor(labels).cuda().long()

                    torch.manual_seed(seed)
                    if use_cuda:
                        torch.cuda.manual_seed(seed)

                    nfeatures=inputAll.shape[1]

                    model = modelsCNN.FC_l3(nfeatures,fc_dim1,fc_dim2,fc_dim3,nclasses,0.5,regrs=regrs)
                    if use_cuda:
                        model.cuda()

                    model.load_state_dict(torch.load(os.path.join(modelsavepath,patientID+'_'+str(testepoch)+'.pt')))
                    with torch.no_grad():
                        model.cuda()
                        model.eval()
                        ig = IntegratedGradients(model)
                        attributions= ig.attribute(inputAll_t, baselines=inputBaseline_t,target=torch.tensor(labels[sampleIdx]).cuda())
                        igRes[sampleIdx]+=attributions.cpu().numpy()

                        pred = model(inputAll_t)
                        attributions= ig.attribute(inputAll_t, baselines=inputBaseline_t,target=torch.argmax(pred,axis=1))
                        igResPred[sampleIdx]+=attributions.cpu().numpy()
            igRes=igRes/inputBaseline.shape[0]
            igResPred=igResPred/inputBaseline.shape[0]
            with open(os.path.join(plotsavepath,'igResControl_'+str(testepoch)), 'wb') as output:
                pickle.dump(igRes,output,pickle.HIGHEST_PROTOCOL)
            with open(os.path.join(plotsavepath,'igResControlPred_'+str(testepoch)), 'wb') as output:
                pickle.dump(igResPred,output,pickle.HIGHEST_PROTOCOL)


APOE
seed:  4
spatial  300
spatial  650
spatial  1000
seed:  8
spatial  300
spatial  650
spatial  1000
seed:  9
spatial  300
spatial  650
spatial  1000
seed:  10
spatial  300
spatial  650
spatial  1000
seed:  11
spatial  300
spatial  650
spatial  1000
seed:  12
spatial  300
spatial  650
spatial  1000
